In [1]:
from main import main
from utils import LoadData, LoadModel, load_data
import argparse
import numpy as np
import time
import copy
import os
import tempfile
from tabulate import tabulate 

import pandas as pd
# pyomo for optimization
import pyomo.environ as pyo

# pytorch for training neural network
import torch.onnx
from torch import nn, optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms

# omlt for interfacing our neural network with pyomo
import onnx
from omlt import OffsetScaling, OmltBlock
from omlt.io.onnx import (
    load_onnx_neural_network_with_bounds,
    write_onnx_model_with_bounds,
    load_onnx_neural_network,
)
from omlt.neuralnet import (FullSpaceNNFormulation, 
    ReluComplementarityFormulation, 
    ReluPartitionFormulation,
    ReducedSpaceNNFormulation)

2025-01-13 20:09:47.933316: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-13 20:09:47.947521: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736820587.960645  794646 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736820587.964519  794646 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-13 20:09:47.979763: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [46]:
def add_arguments(model,model_id,system,scenario):    
    if system == 'PFR':
        args = argparse.Namespace(
            input_dim=5,
            hidden_dim=25,
            hidden_num=3,
            z0_dim=5,
            optimizer='adam',
            epochs=1000,
            batch_size=16,
            lr=1e-4,
            mu=1,
            max_subiter=500,
            eta=0.8,
            sigma=2,
            mu_safe=1e+9,
            dtype=32,
            dataset_path='/home/andresfel9403/KKThNN/KKThPINN/benchmark_PFR.csv',
            val_ratio=0.2,
            job='train',
            runs=10)
    else:
        pass

    args.model = model
    args.model_id = model_id
    args.dataset_type = system
    args.scenario = scenario
        
    
    if args.model == 'NN':
        args.loss_type = 'MSE'
    elif args.model == 'PINN':
        args.loss_type = 'PINN'
    elif args.model == 'KKThPINN':
        args.loss_type = 'MSE'
    elif args.model == 'AugLagNN':
        args.loss_type = 'MSE'
    elif args.model == 'ECNN':
        args.loss_type = 'MSE'
    return args

In [28]:
#columns = inputs+outputs

#df = pd.read_csv("benchmark_membrane.csv", usecols = columns)


## Neural Network Training  

In [47]:
args_KKT = add_arguments('KKThPINN','PFR_KKThPINN','PFR','demonstration')

In [ ]:
main(args_KKT)

After defining the arguments depending on the number of inputs, hidden and output layers, we can train the model selecting the Neural Network model (NN or KKThPINN)

In [48]:
dataset, scaling = load_data(args_KKT.dataset_path)
inputs = dataset[:,:5]
outputs = dataset[:,5:]

data = LoadData(args_KKT)
model_KKT = LoadModel(args_KKT, data)
if args_KKT.job == 'train':
    PATH = '/home/andresfel9403/KKThNN/KKThPINN/models/PFR/KKThPINN/0.2/PFR_KKThPINN_0.2_0.pth'
elif args_KKT.job == 'experiment':
    PATH = '/home/andresfel9403/KKThNN/KKThPINN/models/PFR/KKThPINN/0.2/PFR_KKThPINN_0.2_9.pth'
checkpoint = torch.load(PATH)
model_KKT.load_state_dict(checkpoint['state_dict'])
#model_NN.eval()



type of A: torch.float32, type of B: torch.float32, type of b: torch.float32
train set size: 601, val set size: 200, test set size: 200


/tmp/ipykernel_794646/1431590260.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(PATH)


<All keys matched successfully>

In [49]:
print(PATH)

/home/andresfel9403/KKThNN/KKThPINN/models/PFR/KKThPINN/0.2/PFR_KKThPINN_0.2_0.pth


In [50]:
x_factor = scaling.scale_[:5]
y_factor = scaling.scale_[5:]

print(x_factor)

[1.52711440e+03 1.03091311e+03 1.39930024e+03 4.26720000e+02
 2.20000000e+06]


In [51]:
scaler = scaler = OffsetScaling(
    offset_inputs={i: 0 for i in range(len(x_factor))},
    factor_inputs={i: x_factor[i] for i in range(len(x_factor))},
    offset_outputs={i: 0 for i in range(len(y_factor))},
    factor_outputs={i: y_factor[i] for i in range(len(y_factor))},
)

In [52]:
x_dummy = torch.from_numpy(inputs[0]).float()
ub = np.max(inputs, 0)
lb = np.min(inputs, 0)

scaled_input_bounds = {i: (lb[i], ub[i]) for i in range(len(inputs[0]))}
print(lb, ub)
print(x_dummy)

[0.04081633 0.         0.         0.2979237  0.59090909] [1. 1. 1. 1. 1.]
tensor([0.9490, 0.0907, 0.0111, 0.6093, 0.8264])


In [53]:
def transfer_weights(original_model, modified_model):
    with torch.no_grad():
        for original_layer, modified_layer in zip(original_model.layers, modified_model.layers):
            if isinstance(original_layer, nn.Linear) and isinstance(modified_layer, nn.Linear):
                modified_layer.weight.copy_(original_layer.weight)
                modified_layer.bias.copy_(original_layer.bias)

In [54]:
def _create_onnx_model(data,args,model,file_path,x,input_bounds):
    args.model='NN'
    print('Saving standard model')
    modified_model = LoadModel(args,data)
    transfer_weights(model,modified_model)
    create_onnx_model(data,modified_model,file_path,x,input_bounds)

In [55]:
def create_onnx_model(data,model,file_path,x,input_bounds):
    
    torch.onnx.export(
        model,
        x,
        file_path,
        input_names=["input"],
        dynamo = False
    )
    write_onnx_model_with_bounds(file_path, None, input_bounds)
    print(f"Wrote PyTorch Onnx model to {file_path}")

In [57]:
_create_onnx_model(data,args_KKT,model_KKT,'KKThPINN_PFR.onnx',x_dummy,scaled_input_bounds)

Saving standard model
Wrote PyTorch Onnx model to KKThPINN_PFR.onnx


In [58]:
network_definition_NN = load_onnx_neural_network_with_bounds('KKThPINN_PFR.onnx') 


In [59]:
formulation_NN = FullSpaceNNFormulation(network_definition_NN)

In [60]:
for layer_id, layer in enumerate(network_definition_NN.layers):
    print(f"{layer_id}\t{layer}\t{layer.activation}")

0	InputLayer(input_size=[5], output_size=[5])	linear
1	DenseLayer(input_size=[5], output_size=[25])	relu
2	DenseLayer(input_size=[25], output_size=[25])	relu
3	DenseLayer(input_size=[25], output_size=[25])	relu
4	DenseLayer(input_size=[25], output_size=[5])	linear


In [61]:
inputs = ['F_MeOH',
    'F_DME',
    'F_H2O',
    'T_in',
    'P_in',
]

outputs = [
    'F_MeOH_O',
    'F_DME_O',
    'F_H2O_O',
    'T_Out',
    'P_Out'
]
model = pyo.ConcreteModel()

# create an OMLT block for the neural network and build its formulation
model.membrane= OmltBlock()
model.membrane.build_formulation(formulation_NN)



WARNING (W1002): Setting Var 'membrane.scaled_inputs[0]' to a numeric value
`0` outside the bounds (0.04081632653061225, 1.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'membrane.scaled_inputs[0]' to a numeric value
`0` outside the bounds (0.04081632653061225, 1.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'membrane.scaled_inputs[3]' to a numeric value
`0` outside the bounds (0.29792369703787025, 1.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'membrane.scaled_inputs[4]' to a numeric value
`0` outside the bounds (0.5909090909090909, 1.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002


In [62]:
for i in range(len(inputs)):
    model.membrane.inputs[i] = x_dummy[i].item()
    model.membrane.inputs[i].setlb(lb[i])
    model.membrane.inputs[i].setub(ub[i])
    print(pyo.value(model.membrane.inputs[i].lb)*x_factor[i],pyo.value(model.membrane.inputs[i].ub)*x_factor[i])

for i in range(len(outputs)):
    model.membrane.outputs[i] = 0.0
    model.membrane.outputs[i].setlb(0.0)

62.3312 1527.1144
0.0 1030.913109
0.0 1399.300241
127.13 426.72
1300000.0 2200000.0


In [63]:
Tin_idx = inputs.index('T_in')
F_MeOH_idx = inputs.index('F_MeOH')
F_DME_idx = inputs.index('F_DME')
F_H2O_idx = inputs.index('F_H2O')
P_in_idx = inputs.index('P_in')

FMeOh_O_idx = outputs.index('F_MeOH_O')
FDME_O_idx = outputs.index('F_DME_O')
FH2O_O_idx = outputs.index('F_H2O_O')
TOut_idx = outputs.index('T_Out')
POut_idx = outputs.index('P_Out')

# def Total_Flow(m):
#     FMeOh = m.membrane.inputs[F_MeOH_idx]*x_factor[F_MeOH_idx]
#     FDME = m.membrane.inputs[F_DME_idx]*x_factor[F_DME_idx]
#     FH2O = m.membrane.inputs[F_H2O_idx]*x_factor[F_H2O_idx]
#     Ftot = FMeOh*0.93 + FDME*0.06 + FH2O*0.01
#     return Ftot


def Tin_bounds(m):
    return (460,m.membrane.inputs[Tin_idx]*x_factor[Tin_idx],1000)

def Tout_bounds(m):
    return (460,y_factor[TOut_idx]*m.membrane.outputs[TOut_idx],1000)



def TFlow_bounds(m):
    FMeOh = m.membrane.inputs[F_MeOH_idx]*x_factor[F_MeOH_idx]
    FDME = m.membrane.inputs[F_DME_idx]*x_factor[F_DME_idx]
    FH2O = m.membrane.inputs[F_H2O_idx]*x_factor[F_H2O_idx]
    Ftot = FMeOh + FDME + FH2O 
    return (0, Ftot, None)


def P_bounds(m):
    return (13e5, m.membrane.inputs[P_in_idx]*x_factor[P_in_idx], 27e5)

def POut_bounds(m):
    return (13e5, m.membrane.outputs[POut_idx]*y_factor[POut_idx], 27e5)

def Mas_Bal(m):
    MW = {
        "MeOH": 32.04,  # MW of Methanol in g/mol
        "DME": 46.069,   # Cost of Dimethyl Ether in $/kg
        "H2O": 18.0152,  # Cost of Water in $/kg
        "N2": 28.0134  # Cost of Nitrogen in $/kg
    }

    FMeOh = m.membrane.inputs[F_MeOH_idx]*x_factor[F_MeOH_idx]
    FDME = m.membrane.inputs[F_DME_idx]*x_factor[F_DME_idx]
    FH2O = m.membrane.inputs[F_H2O_idx]*x_factor[F_H2O_idx]
    FMeOh_O = m.membrane.outputs[FMeOh_O_idx]*y_factor[FMeOh_O_idx]
    FDME_O = m.membrane.outputs[FDME_O_idx]*y_factor[FDME_O_idx]
    FH2O_O = m.membrane.outputs[FH2O_O_idx]*y_factor[FH2O_O_idx]

    In = FMeOh * MW["MeOH"] + FDME * MW["DME"] + FH2O * MW["H2O"] 
    Out = FMeOh_O * MW["MeOH"] + FDME_O * MW["DME"] + FH2O_O * MW["H2O"] 
    MassBal = abs(In - Out)
    return (0, MassBal, 1e-6)

# def Flow_Nitrog(m):
#     FN2 = m.membrane.inputs[F_N2_idx]*x_factor[F_N2_idx]
#     FN2_O = m.membrane.outputs[FN2_O_idx]*y_factor[FN2_O_idx]
    
#     return (FN2-FN2_O == 0)






#model.Tin_constr = pyo.Constraint(rule = Tin_bounds)
#model.Tm_constr = pyo.Constraint(rule = T_mem_bounds)
#model.TFlow_constr = pyo.Constraint(rule = TFlow_bounds)
#model.P_constr = pyo.Constraint(rule = P_bounds)
model.MassBal = pyo.Constraint(rule = Mas_Bal)
#model.Flow_Nitrog = pyo.Constraint(rule = Flow_Nitrog)
#model.Tout_constr = pyo.Constraint(rule = Tout_bounds)
#model.Tout_M_constr = pyo.Constraint(rule = TOut_mem_bounds)


#model.POut_constr = pyo.Constraint(rule = POut_bounds)

model.Total_Flow = pyo.Var(bounds=(1, 2000))

# Add a constraint to enforce the relationship
def Total_Flow_Constraint(m):
    FMeOh = m.membrane.inputs[F_MeOH_idx] * x_factor[F_MeOH_idx]
    FDME = m.membrane.inputs[F_DME_idx] * x_factor[F_DME_idx]
    FH2O = m.membrane.inputs[F_H2O_idx] * x_factor[F_H2O_idx]
    return m.Total_Flow == FMeOh * 0.93 + FDME * 0.06 + FH2O * 0.01

model.Total_Flow_Constraint = pyo.Constraint(rule=Total_Flow_Constraint)





In [36]:
def objective_rule(m):
    prices = {
        "MeOH": 891*1e-6,  # Cost of Methanol in $/kg
        "DME": 2021.84*1e-6,   # Cost of Dimethyl Ether in $/kg
        "H2O": 0.29*1e-6,  # Cost of Water in $/kg
        "N2": 121.254*1e-6  # Cost of Nitrogen in $/kg
    }

    MW = {
        "MeOH": 32.04,  # MW of Methanol in g/mol
        "DME": 46.069,   # Cost of Dimethyl Ether in $/kg
        "H2O": 18.0152,  # Cost of Water in $/kg
        "N2": 28.0134  # Cost of Nitrogen in $/kg
    }

    DME_O = m.membrane.outputs[outputs.index("F_DME_O")]*y_factor[outputs.index("F_DME_O")]
    H2O_O = m.membrane.outputs[outputs.index("F_H2O_O")]*y_factor[outputs.index("F_H2O_O")]
    MeOH_In = m.membrane.inputs[inputs.index("F_MeOH")]*x_factor[inputs.index("F_MeOH")]
    DME_In = m.membrane.inputs[inputs.index("F_DME")]*x_factor[inputs.index("F_DME")]

    product_profit = prices["DME"]*DME_O * MW["DME"] + prices["H2O"]*(H2O_O) * MW["H2O"]

    feedstock_cost = prices["MeOH"]*MeOH_In * MW["MeOH"] + prices["DME"]*DME_O * MW["DME"]
    return product_profit - feedstock_cost

model.objective = pyo.Objective(rule = objective_rule, sense = pyo.maximize)


'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [37]:
solver = pyo.SolverFactory("ipopt")
status = solver.solve(model, tee=True)

Ipopt 3.14.6: 
==> Warning: Treating 75 binary and 0 integer variables as continuous.


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     1559
Number of nonzeros in inequality constraint Jacobian.:      594
Number of nonzeros in Lagrangian Hessian.............:       21

Total number of variables............................:      258
                     variables with only lower bounds:        5
                variables with lower and upper bounds:      248
                     variables with only upper bounds:        0
Total 

In [38]:
F_MEOH_IN = pyo.value(model.membrane.inputs[inputs.index("F_MeOH")])*x_factor[inputs.index("F_MeOH")]
F_MEOH_OUT = pyo.value(model.membrane.outputs[outputs.index("F_MeOH_O")])*y_factor[outputs.index("F_MeOH_O")]

F_DEM_IN = pyo.value(model.membrane.inputs[inputs.index("F_DME")])*x_factor[inputs.index("F_DME")]
F_DME_OUT = pyo.value(model.membrane.outputs[outputs.index("F_DME_O")])*y_factor[outputs.index("F_DME_O")]

F_H2O_IN = pyo.value(model.membrane.inputs[inputs.index("F_H2O")])*x_factor[inputs.index("F_H2O")]
F_H2O_OUT = pyo.value(model.membrane.outputs[outputs.index("F_H2O_O")])*y_factor[outputs.index("F_H2O_O")]

T_IN = pyo.value(model.membrane.inputs[inputs.index("T_in")])*x_factor[inputs.index("T_in")]
T_OUT = pyo.value(model.membrane.outputs[outputs.index("T_Out")])*y_factor[outputs.index("T_Out")]


P_IN = pyo.value(model.membrane.inputs[inputs.index("P_in")])*x_factor[inputs.index("P_in")]
P_OUT = pyo.value(model.membrane.outputs[outputs.index("P_Out")])*y_factor[outputs.index("P_Out")]

CONVERSION = (F_MEOH_IN - F_MEOH_OUT)/F_MEOH_IN
Mas_Bal = pyo.value(model.MassBal)

data_tabulate = [
    ["Input Parameter","Value","Output Parameter","Value"],
    ["F_MeOH", F_MEOH_IN, "F_MeOH_O", F_MEOH_OUT],
    ["F_DME", F_DEM_IN, "F_DME_O", F_DME_OUT],
    ["F_H2O", F_H2O_IN, "F_H2O_O", F_H2O_OUT],
    ["T_in", T_IN, "T_Out", T_OUT],
    ["P_in", P_IN, "P_Out", P_OUT],
]

conversion_table = [
    ["MeOH Conversion", CONVERSION]
]

print(tabulate(data_tabulate, headers="firstrow", tablefmt="grid", colalign=("left", "left", "left", "left")))
print(tabulate(conversion_table, tablefmt="grid", colalign=("left", "left")))
print(pyo.value(model.Total_Flow))
print(f"Mass Balance Violation: {Mas_Bal}")

+-------------------+-------------+--------------------+-------------+
| Input Parameter   | Value       | Output Parameter   | Value       |
+===================+=============+====================+=============+
| F_MeOH            | 62.3312     | F_MeOH_O           | 53.7668     |
+-------------------+-------------+--------------------+-------------+
| F_DME             | 0.0272082   | F_DME_O            | 0.0014366   |
+-------------------+-------------+--------------------+-------------+
| F_H2O             | 1073.7      | F_H2O_O            | 1088.99     |
+-------------------+-------------+--------------------+-------------+
| T_in              | 426.708     | T_Out              | 594.633     |
+-------------------+-------------+--------------------+-------------+
| P_in              | 1.30037e+06 | P_Out              | 3.01298e+06 |
+-------------------+-------------+--------------------+-------------+
+-----------------+----------+
| MeOH Conversion | 0.137401 |
+--------------